In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [3]:
import keras
keras.__version__

'2.4.3'

In [4]:
!pip install tf2crf

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [6]:
import pandas as pd

In [7]:
X_train, X_test, y_train, y_test = pd.read_pickle('data/ner_kor_dataset.pkl')

In [8]:
print('훈련 샘플 문장의 크기 : {}'.format(X_train.shape))
print('훈련 샘플 레이블의 크기 : {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기 : {}'.format(X_test.shape))
print('테스트 샘플 레이블의 크기 : {}'.format(y_test.shape))

훈련 샘플 문장의 크기 : (14964, 125)
훈련 샘플 레이블의 크기 : (14964, 125, 22)
테스트 샘플 문장의 크기 : (3742, 125)
테스트 샘플 레이블의 크기 : (3742, 125, 22)


In [9]:
vocab_size = 41640
tag_size = 22
max_len = 125

---

In [10]:
from tf2crf import CRF
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
# from keras_contrib.layers import CRF

In [11]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=20, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1)))
model.add(TimeDistributed(Dense(50, activation="relu")))  #각 스텝마다 cost (오류)를 계산해서 하위 스텝으로 오류를 전파
crf = CRF(tag_size)
model.add(crf)

In [12]:
model.compile(optimizer="rmsprop", loss=crf.loss
              , metrics=[crf.accuracy])
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Epoch 1/10
421/421 [==============================] - 97s 231ms/step - loss: 6.3119 - accuracy: 0.8004 - val_loss: 15.5312 - val_accuracy: 0.8371
Epoch 2/10
421/421 [==============================] - 95s 226ms/step - loss: 3.6692 - accuracy: 0.8438 - val_loss: 12.2606 - val_accuracy: 0.8512
Epoch 3/10
421/421 [==============================] - 94s 224ms/step - loss: 2.8524 - accuracy: 0.8557 - val_loss: 9.5545 - val_accuracy: 0.8616
Epoch 4/10
421/421 [==============================] - 94s 224ms/step - loss: 2.2908 - accuracy: 0.8726 - val_loss: 7.6769 - val_accuracy: 0.8745
Epoch 5/10
421/421 [==============================] - 95s 226ms/step - loss: 1.9310 - accuracy: 0.8874 - val_loss: 6.7923 - val_accuracy: 0.8831
Epoch 6/10
421/421 [==============================] - 94s 224ms/step - loss: 1.6857 - accuracy: 0.8971 - val_loss: 6.5672 - val_accuracy: 0.8883
Epoch 7/10
421/421 [==============================] - 95s 225ms/step - loss: 1.5099 - accuracy: 0.9053 - val_loss: 6.7826 - val_

In [13]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

117/117 [==============================] - 12s 103ms/step - loss: 8.6431 - accuracy: 0.9005

 테스트 정확도: 0.9005
